# 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%matplotlib inline

# 데이터 확인

In [2]:
# 데이터의 경로 각자에 맞추어서 변경해주세요
data_dir = '/content/drive/MyDrive/앙상블 강의자료/과제/'

train = pd.read_csv(data_dir + 'train.csv')
test = pd.read_csv(data_dir + 'test.csv')
submission = pd.read_csv(data_dir + 'sample_submission.csv')

In [3]:
train.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
0,44,F,1,Unknown,Divorced,Unknown,Blue,36,3,2,1,17116.0,1289,15827.0,0.559,3632,50,0.923,0.075,0
1,42,M,3,Uneducated,Single,$60K - $80K,Blue,30,4,2,3,8302.0,1294,7008.0,0.558,3221,66,0.610,0.156,0
2,57,M,0,Uneducated,Divorced,$80K - $120K,Blue,39,4,4,3,2795.0,1832,963.0,0.839,4679,67,0.861,0.655,0
3,26,F,1,Graduate,Single,$40K - $60K,Blue,20,6,3,3,1438.3,673,765.3,0.595,2167,48,0.412,0.468,0
4,38,F,0,Uneducated,Married,Less than $40K,Blue,26,5,1,4,2230.0,0,2230.0,0.896,3766,63,0.909,0.000,0


In [4]:
test.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,53,F,2,College,Single,Less than $40K,Blue,45,6,3,3,6210.0,1083,5127.0,0.621,3095,62,0.590,0.174
1,52,M,1,College,Married,$80K - $120K,Blue,48,4,3,3,2066.0,1601,465.0,0.742,1578,41,0.414,0.775
2,27,M,0,Unknown,Married,$40K - $60K,Blue,14,4,1,4,3823.0,2130,1693.0,0.371,2026,49,0.441,0.557
3,43,M,3,Unknown,Single,$60K - $80K,Blue,34,6,2,0,13771.0,2517,11254.0,0.501,1510,42,0.750,0.183
4,55,F,2,High School,Married,$40K - $60K,Blue,42,2,2,3,5431.0,2007,3424.0,0.661,4569,82,0.608,0.370


In [5]:
submission.head()

,id,Existing Customer,Attrited Customer
0,0,0,1
1,1,0,1
2,2,0,1
3,3,0,1
4,4,0,1


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8101 entries, 0 to 8100
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_Age              8101 non-null   int64  
 1   Gender                    8101 non-null   object 
 2   Dependent_count           8101 non-null   int64  
 3   Education_Level           8101 non-null   object 
 4   Marital_Status            8101 non-null   object 
 5   Income_Category           8101 non-null   object 
 6   Card_Category             8101 non-null   object 
 7   Months_on_book            8101 non-null   int64  
 8   Total_Relationship_Count  8101 non-null   int64  
 9   Months_Inactive_12_mon    8101 non-null   int64  
 10  Contacts_Count_12_mon     8101 non-null   int64  
 11  Credit_Limit              8101 non-null   float64
 12  Total_Revolving_Bal       8101 non-null   int64  
 13  Avg_Open_To_Buy           8101 non-null   float64
 14  Total_Am

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_Age              2026 non-null   int64  
 1   Gender                    2026 non-null   object 
 2   Dependent_count           2026 non-null   int64  
 3   Education_Level           2026 non-null   object 
 4   Marital_Status            2026 non-null   object 
 5   Income_Category           2026 non-null   object 
 6   Card_Category             2026 non-null   object 
 7   Months_on_book            2026 non-null   int64  
 8   Total_Relationship_Count  2026 non-null   int64  
 9   Months_Inactive_12_mon    2026 non-null   int64  
 10  Contacts_Count_12_mon     2026 non-null   int64  
 11  Credit_Limit              2026 non-null   float64
 12  Total_Revolving_Bal       2026 non-null   int64  
 13  Avg_Open_To_Buy           2026 non-null   float64
 14  Total_Am

# BaseLine Model

저는 라벨인코딩만을 이용한 간단한 베이스 라인 모델을 만들겠습니다.

16기 분들은 feature engineering, 하이퍼 파라미터 튜닝, 앙상블 등 다양한 방법으로 성능이 우수한 모델을 만들어보세요!

## 라벨인코딩

In [8]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook

cat_cols = test.dtypes[test.dtypes == 'O'].index.tolist()

lbl = LabelEncoder()

# 원본데이터의 변형을 막고자 데이터를 copy함
temp_train = train.copy()
temp_test = test.copy()

for col in tqdm_notebook( cat_cols ):
  # local_train과 local_test를 concat하여 temp_df에 저장
  temp_df = pd.concat([ temp_train[[col]], temp_test[[col]] ] , axis=0)
  
  # Label-Encoding을 fitting함
  lbl.fit( temp_df[col] )
  
  # local_train/local_test에 label_encoding한 값을 대입함
  temp_train[col] = lbl.transform(temp_train[col])
  temp_test[col] = lbl.transform(temp_test[col])

features = test.columns.tolist()
target = 'Attrition_Flag'

X_train = temp_train[features]
y_train = temp_train[target]

X_test = temp_test[features]

In [9]:
X_train.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,44,0,1,6,0,5,0,36,3,2,1,17116.0,1289,15827.0,0.559,3632,50,0.923,0.075
1,42,1,3,5,2,2,0,30,4,2,3,8302.0,1294,7008.0,0.558,3221,66,0.610,0.156
2,57,1,0,5,0,3,0,39,4,4,3,2795.0,1832,963.0,0.839,4679,67,0.861,0.655
3,26,0,1,2,2,1,0,20,6,3,3,1438.3,673,765.3,0.595,2167,48,0.412,0.468
4,38,0,0,5,1,4,0,26,5,1,4,2230.0,0,2230.0,0.896,3766,63,0.909,0.000


In [10]:
X_test.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,53,0,2,0,2,4,0,45,6,3,3,6210.0,1083,5127.0,0.621,3095,62,0.590,0.174
1,52,1,1,0,1,3,0,48,4,3,3,2066.0,1601,465.0,0.742,1578,41,0.414,0.775
2,27,1,0,6,1,1,0,14,4,1,4,3823.0,2130,1693.0,0.371,2026,49,0.441,0.557
3,43,1,3,6,2,2,0,34,6,2,0,13771.0,2517,11254.0,0.501,1510,42,0.750,0.183
4,55,0,2,3,1,1,0,42,2,2,3,5431.0,2007,3424.0,0.661,4569,82,0.608,0.370


## 성능 평가를 위한 데이터셋 분리

oof 방식을 통해서 데이터의 성능 평가를 해도 무방합니다. <- 제가 제일 선호하는 방식입니다,

저는 16기 분들의 이해를 돕기 위해 먼저 Train과 Val data로 나누었습니다.

In [11]:
y_train.value_counts() / y_train.value_counts().sum()

0    0.839279
1    0.160721
Name: Attrition_Flag, dtype: float64

데이터 라벨이 불균형하기 때문에 stratify 하게 데이터를 분리하겠습니다.

In [12]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X_train, y_train, test_size=0.2, random_state = 16, stratify = y_train)

In [13]:
train_y.value_counts() / train_y.value_counts().sum()

0    0.839352
1    0.160648
Name: Attrition_Flag, dtype: float64

In [14]:
val_y.value_counts() / val_y.value_counts().sum()

0    0.838988
1    0.161012
Name: Attrition_Flag, dtype: float64

데이터가 잘 분리되었다는 것을 확인할 수 있습니다.

## Voting

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

log_clf = LogisticRegression(random_state = 16)
dt_clf = DecisionTreeClassifier(random_state = 16, max_depth = None)
svm_clf = SVC(probability = True, random_state = 16) # 클래스의 예측 확률을 구하기 위해서

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('dt', dt_clf), ('svm', svm_clf)],
    voting = 'soft'
)

In [16]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

for clf in [log_clf, dt_clf, svm_clf, voting_clf]:
  clf.fit(train_X, train_y)
  y_pred = clf.predict_proba(val_X)
  print(clf.__class__.__name__,'Logloss Score:' , log_loss(val_y, y_pred))
  print(clf.__class__.__name__,'Accuracy Score:' , accuracy_score(val_y, np.argmax(y_pred, axis = 1)))
  print()

LogisticRegression Logloss Score: 0.28011639992085835
LogisticRegression Accuracy Score: 0.878470080197409

DecisionTreeClassifier Logloss Score: 2.3650858604781537
DecisionTreeClassifier Accuracy Score: 0.9315237507711289

SVC Logloss Score: 0.3143435717055473
SVC Accuracy Score: 0.879086983343615

VotingClassifier Logloss Score: 0.20551390156353488
VotingClassifier Accuracy Score: 0.9222702035780382



Voting의 성능이 가장 좋다는 것을 확인할 수 있습니다. 

하지만 단일의 DecisionTreeClassifier 의 성능은 매우 좋지 않네요


## Bagging and Pasting

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

dt_clf2 = DecisionTreeClassifier(random_state = 16, max_depth = None)

bag_clf = BaggingClassifier(dt_clf2,
                            n_estimators = 500,
                            bootstrap = True, n_jobs = -1) # Bagging

pas_clf = BaggingClassifier(dt_clf2,
                            n_estimators = 500,
                            bootstrap = False, n_jobs = -1) # Pasting

In [18]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

for clf in [bag_clf, pas_clf]:
  clf.fit(train_X, train_y)
  y_pred = clf.predict_proba(val_X)
  print(clf.__class__.__name__,'Logloss Score:' , log_loss(val_y, y_pred))
  print(clf.__class__.__name__,'Accuracy Score:' , accuracy_score(val_y, np.argmax(y_pred, axis = 1)))
  print()

BaggingClassifier Logloss Score: 0.10566237954652154
BaggingClassifier Accuracy Score: 0.9673041332510796

BaggingClassifier Logloss Score: 1.5112091518258481
BaggingClassifier Accuracy Score: 0.933374460209747



Bagging 방식이 성능이 더 좋다는 것을 확인 할 수 있습니다.

그리고 두 방식 모두 단일의 DecisionTreeClassifier 보다 성능이 더 높게 나온다는 것도 확인할 수 있습니다.

## Random Patches Method and Random Subspace Method

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

dt_clf2 = DecisionTreeClassifier(random_state = 16, max_depth = None)

pas_rpm_clf = BaggingClassifier(dt_clf2,
                            n_estimators = 500,
                            bootstrap = False, max_samples = 1.0,
                            bootstrap_features = False, max_features = 1.0, 
                            n_jobs = -1) # Pasting and Random Patches Method (위와 동일한 모델임)

bag_rsm_clf = BaggingClassifier(dt_clf2,
                            n_estimators = 500,
                            bootstrap = True, max_samples = 1.0,
                            bootstrap_features = True, max_features = 0.8, 
                            n_jobs = -1) # Bagging and Random Subspace Method

In [20]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

for clf in [pas_rpm_clf, bag_rsm_clf]:
  clf.fit(train_X, train_y)
  y_pred = clf.predict_proba(val_X)
  print(clf.__class__.__name__,'Logloss Score:' , log_loss(val_y, y_pred))
  print(clf.__class__.__name__,'Accuracy Score:' , accuracy_score(val_y, np.argmax(y_pred, axis = 1)))
  print()

BaggingClassifier Logloss Score: 1.5108442858648425
BaggingClassifier Accuracy Score: 0.9352251696483652

BaggingClassifier Logloss Score: 0.14725677361912318
BaggingClassifier Accuracy Score: 0.9636027143738433



## RandomForest

In [21]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state = 16, n_estimators = 500, n_jobs = -1)

In [22]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

rf_clf.fit(train_X, train_y)
y_pred = rf_clf.predict_proba(val_X)
print(rf_clf.__class__.__name__,'Logloss Score:' , log_loss(val_y, y_pred))
print(rf_clf.__class__.__name__,'Accuracy Score:' , accuracy_score(val_y, np.argmax(y_pred, axis = 1)))

RandomForestClassifier Logloss Score: 0.12335728190343818
RandomForestClassifier Accuracy Score: 0.9623689080814312


단일의 DecisionTreeClassifier 보다 Bagging을 활용한 RandomForestClassifier 모델이 성능이 더 높다는 것을 확인할 수 있습니다.

## ExtraTrees

In [23]:
from sklearn.ensemble import ExtraTreesClassifier

et_clf = ExtraTreesClassifier(random_state = 16, n_estimators = 500, n_jobs = -1)

In [24]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

et_clf.fit(train_X, train_y)
y_pred = et_clf.predict_proba(val_X)
print(et_clf.__class__.__name__,'Logloss Score:' , log_loss(val_y, y_pred))
print(et_clf.__class__.__name__,'Accuracy Score:' , accuracy_score(val_y, np.argmax(y_pred, axis = 1)))

ExtraTreesClassifier Logloss Score: 0.16195814884097223
ExtraTreesClassifier Accuracy Score: 0.9444787168414559


## Adaboost

In [25]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

dt_clf2 = DecisionTreeClassifier(random_state = 16, max_depth = 1)

ada_clf = AdaBoostClassifier(base_estimator = dt_clf2,
                             n_estimators = 500,
                             learning_rate = 0.1,
                             random_state = 16)

In [26]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

ada_clf.fit(train_X, train_y)
y_pred = ada_clf.predict_proba(val_X)
print(ada_clf.__class__.__name__,'Logloss Score:' , log_loss(val_y, y_pred))
print(ada_clf.__class__.__name__,'Accuracy Score:' , accuracy_score(val_y, np.argmax(y_pred, axis = 1)))

AdaBoostClassifier Logloss Score: 0.6389401559598495
AdaBoostClassifier Accuracy Score: 0.9605181986428131


## Gradient Boosting

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.1, random_state = 16)

In [28]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

gb_clf.fit(train_X, train_y)
y_pred = gb_clf.predict_proba(val_X)
print(gb_clf.__class__.__name__,'Logloss Score:' , log_loss(val_y, y_pred))
print(gb_clf.__class__.__name__,'Accuracy Score:' , accuracy_score(val_y, np.argmax(y_pred, axis = 1)))

GradientBoostingClassifier Logloss Score: 0.06951418672974169
GradientBoostingClassifier Accuracy Score: 0.974090067859346


번외로 부스팅 모델의 잔차학습 방식을 회귀 데이터 셋으로 구현해보겠습니다.

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

X, y = make_regression(n_samples = 10000, n_features = 10, noise = 3, random_state = 16) # 회귀 데이터셋 생성

boosting_train_X, boosting_val_X, boosting_train_y, boosting_val_y = train_test_split(X, y, test_size=0.2, random_state = 16)

# First Model

base_tree = DecisionTreeRegressor(max_depth = 3, random_state = 16)

base_tree.fit(boosting_train_X, boosting_train_y)

F_new = np.mean(base_tree.predict(boosting_train_X))

pred_F_new = np.mean(base_tree.predict(boosting_val_X))

lr = 0.1 # 학습률

# Iteration
for i in range(100):
  F_old = F_new
  pred_F_old = pred_F_new

  r = boosting_train_y - F_old # 잔차 구하기

  base_tree.fit(boosting_train_X, r) # 잔차를 예측하는 모델 학습

  gamma = base_tree.predict(boosting_train_X) # 잔차 예측
  pred_gamma = base_tree.predict(boosting_val_X)

  F_new = F_old + lr * gamma # 새로운 예측값

  pred_F_new = pred_F_old + lr * pred_gamma

  print(f'{i + 1}개의 모델의 잔차 학습 결과 Train Data : {mean_squared_error(boosting_train_y, F_new)}')
  print(f'{i + 1}개의 모델의 잔차 학습 결과 Test Data  : {mean_squared_error(boosting_val_y, pred_F_new)}\n')

1개의 모델의 잔차 학습 결과 Train Data : 26103.803579430813
1개의 모델의 잔차 학습 결과 Test Data  : 26031.840442680295

2개의 모델의 잔차 학습 결과 Train Data : 24064.570279896216
2개의 모델의 잔차 학습 결과 Test Data  : 24096.24089513111

3개의 모델의 잔차 학습 결과 Train Data : 22330.724749436788
3개의 모델의 잔차 학습 결과 Test Data  : 22417.59430242647

4개의 모델의 잔차 학습 결과 Train Data : 20777.431986547555
4개의 모델의 잔차 학습 결과 Test Data  : 20839.799546304475

5개의 모델의 잔차 학습 결과 Train Data : 19425.051533182523
5개의 모델의 잔차 학습 결과 Test Data  : 19523.114215419555

6개의 모델의 잔차 학습 결과 Train Data : 18197.914743538717
6개의 모델의 잔차 학습 결과 Test Data  : 18355.98558744592

7개의 모델의 잔차 학습 결과 Train Data : 17065.15792225388
7개의 모델의 잔차 학습 결과 Test Data  : 17232.841945457825

8개의 모델의 잔차 학습 결과 Train Data : 16064.297392743145
8개의 모델의 잔차 학습 결과 Test Data  : 16329.470578924234

9개의 모델의 잔차 학습 결과 Train Data : 15134.625945925423
9개의 모델의 잔차 학습 결과 Test Data  : 15459.640422679982

10개의 모델의 잔차 학습 결과 Train Data : 14287.775317597867
10개의 모델의 잔차 학습 결과 Test Data  : 14623.363890831057

11개의 모델의 잔차 

## eXtreme Gradient Boosting and Light Gradient Boosting Machine

이밖에도 Catboost, EBM 등 다양한 모델이 존재합니다!

본 모델들은 eval set을 활용하면 성능이 더 높게 올라갈 수 있습니다!

In [30]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

xgb_clf = XGBClassifier(random_state = 16,
                        n_estimators = 500)

lgbm_clf = LGBMClassifier(random_state = 16,
                          n_estimators = 500)

In [31]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

for clf in [xgb_clf, lgbm_clf]:
  clf.fit(train_X, train_y)
  y_pred = clf.predict_proba(val_X)
  print(clf.__class__.__name__,'Logloss Score:' , log_loss(val_y, y_pred))
  print(clf.__class__.__name__,'Accuracy Score:' , accuracy_score(val_y, np.argmax(y_pred, axis = 1)))
  print()

XGBClassifier Logloss Score: 0.06932638280150479
XGBClassifier Accuracy Score: 0.9728562615669339

LGBMClassifier Logloss Score: 0.12226522993418673
LGBMClassifier Accuracy Score: 0.972239358420728



## Stacking

Meta Model 은 LogisticRegression 으로

개별 모델은 가장 성능이 좋았던 

Bagging, 

RandomForestClassifier, 

GradientBoostingClassifier, 

XGBClassifier, 

LGBMClassifier 

을 사용하겠습니다.

In [32]:
train_pred_li = []
val_pred_li = []
test_pred_li = []

for clf in [bag_clf, rf_clf, gb_clf, xgb_clf, lgbm_clf]:
  clf.fit(train_X, train_y)

  y_train_pred = clf.predict_proba(train_X)
  train_pred_li.append(y_train_pred)

  y_val_pred = clf.predict_proba(val_X)
  val_pred_li.append(y_val_pred)

  y_test_pred = clf.predict_proba(X_test)
  test_pred_li.append(y_test_pred)

In [33]:
stacking_train = np.concatenate(train_pred_li, axis = 1)
stacking_val = np.concatenate(val_pred_li, axis = 1)
stacking_test = np.concatenate(test_pred_li, axis = 1)

In [34]:
stacking_train.shape

(6480, 10)

In [35]:
stacking_val.shape

(1621, 10)

In [36]:
stacking_test.shape

(2026, 10)

In [37]:
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression(random_state = 16)

In [38]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

meta_model.fit(stacking_train, train_y)
y_pred = meta_model.predict_proba(stacking_val)
print(meta_model.__class__.__name__,'Logloss Score:' , log_loss(val_y, y_pred))
print(meta_model.__class__.__name__,'Accuracy Score:' , accuracy_score(val_y, np.argmax(y_pred, axis = 1)))

LogisticRegression Logloss Score: 0.11485661519622652
LogisticRegression Accuracy Score: 0.9753238741517581


집단지성이 무조건 한명의 완벽한 전문가를 이긴다는 보장은 없기 때문에 XGBClassifier, GradientBoostingClassifier 보다는 성능이 낮다는 것을 확인할 수 있습니다.

하지만 대체적으로 대부분의 모델보다는 성능이 좋다는 것 또한 확인할 수 있습니다.

# 예측

최종 예측은 Stacking 을 사용하도록 하겠습니다.

16기 분들은 다양한 방식을 활용해서 모델의 성능을 높여주세요!

In [39]:
y_pred = meta_model.predict_proba(stacking_test)
submission.iloc[:, 1:] = y_pred

In [40]:
submission.to_csv(data_dir + 'Stacking_baseline.csv', index = False)
submission.head()

,id,Existing Customer,Attrited Customer
0,0,0.999720,0.000280
1,1,0.999576,0.000424
2,2,0.985069,0.014931
3,3,0.999628,0.000372
4,4,0.999732,0.000268
